In [ ]:
import os
import shutil
import subprocess

from tqdm.notebook import tqdm

#list all input files

# use preprocessed_data
input_file_dir = "./lpla_trimmed"

input_files = []
skip_files = []
for root, dirs, files in os.walk(input_file_dir, topdown=False):
    for name in files:
        fp = os.path.join(root, name)
        print(fp)

        if fp.endswith(".dat"):
            input_files.append(fp)
        else:
            skip_files.append(fp)

#set output directory
output_dir = "denss_lpla"
os.makedirs(output_dir, exist_ok=True)

#run denss through subprocess
prefixes = []
for file in tqdm(input_files):
    #get file prefix to look for fitdata file later
    file_prefix = os.path.splitext(os.path.basename(file))[0]
    prefixes.append(file_prefix)
    outputs_exist = (file_prefix) in os.listdir(output_dir)

    #run denss
    if not outputs_exist:
        subprocess.check_output(
            f"denss-all -f {file} -m S -j 16 -o {file_prefix}",
            shell=True,
        )
    else:
        print(f"Skipping {file_prefix}, analysis already completed...")

    file_prefix = os.path.splitext(os.path.basename(file))[0]

    #move results to output directory
    shutil.move(
        os.path.join("./", file_prefix), os.path.join("./", output_dir, file_prefix)
    )
    fname = file_prefix + "_fitdata.fit"
    shutil.move(os.path.join("./", fname), os.path.join("./", output_dir, fname))